<a href="https://colab.research.google.com/github/LuchithaP/ML/blob/main/Copy_of_BDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.9

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 42.0 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 24.3.25
    Uninstalling flatbuffers-24.3.25:
      Successfully uninstalled flatbuffers-24.3.25
  Attempting uninstall: tensorboard-data-serve

In [2]:
#df['right/left'] = df.apply(lambda row: 'Left' if row['Alpha_TP9'] > row['Alpha_AF7'] and row['Alpha_TP9'] > row['Alpha_AF8'] else 'Right', axis=1)

In [3]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [4]:
train_path = "/content/train_data.csv"
test_path = "/content/test_data.csv"

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [6]:
FEATURE_NAMES = [
    "Delta_TP9", "Delta_AF7", "Delta_AF8", "Delta_TP10",
    "Theta_TP9", "Theta_AF7", "Theta_AF8", "Theta_TP10",
    "Alpha_TP9", "Alpha_AF7", "Alpha_AF8", "Alpha_TP10",
    "Beta_TP9", "Beta_AF7", "Beta_AF8", "Beta_TP10",
    "Gamma_TP9", "Gamma_AF7", "Gamma_AF8", "Gamma_TP10",
    "RAW_TP9", "RAW_AF7", "RAW_AF8", "RAW_TP10",

]

In [13]:
train = pd.read_csv(train_path)
train_y = train.pop('right/left')
test = pd.read_csv(test_path)
test_y = test.pop('right/left')

In [9]:
# Define the input_fn function
def input_fn(features, labels, batch_size=32, training=True):
    # Convert pandas DataFrame to TensorFlow Dataset
    features = features.fillna(0)
    labels = labels.fillna(0)
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and batch if training, otherwise no shuffle
    if training:
        dataset = dataset.shuffle(buffer_size=1000).batch(batch_size)
    else:
        dataset = dataset.batch(batch_size)

    return dataset

# Assuming train and train_y are already defined and preprocessed as pandas DataFrames

# Create a feature column for the classifier
feature_columns = []
for column in train.columns:
    feature_columns.append(tf.feature_column.numeric_column(key=column))

# Define the classifier model (e.g., DNNClassifier)
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[64, 32],  # Example hidden units for the DNN
    n_classes= 2 # Assuming binary classification (e.g., 'right' or 'left')
)
# Train the classifier
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000
)


In [14]:
eval_result = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))

# Print the evaluation results
print("Evaluation Results:")
print(f"Loss: {eval_result['loss']}")
print(f"Accuracy: {eval_result['accuracy']}")

Evaluation Results:
Loss: 0.46899282932281494
Accuracy: 0.8019890189170837


In [16]:
def input_fn(features, batch_size=256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

# Define your features list
features = [
    "Delta_TP9", "Delta_AF7", "Delta_AF8", "Delta_TP10",
    "Theta_TP9", "Theta_AF7", "Theta_AF8", "Theta_TP10",
    "Alpha_TP9", "Alpha_AF7", "Alpha_AF8", "Alpha_TP10",
    "Beta_TP9", "Beta_AF7", "Beta_AF8", "Beta_TP10",
    "Gamma_TP9", "Gamma_AF7", "Gamma_AF8", "Gamma_TP10",
    "RAW_TP9", "RAW_AF7", "RAW_AF8", "RAW_TP10"
]

# Define a placeholder for predictions
predict = {}

# Define species (map of class_id to label)
SPECIES = {
    0: 'Yes',  # Left corresponds to Yes
    1: 'No'    # Right corresponds to No
}


#print("Please type numeric values as prompted.")

# Loop to get user input for each feature
# This block simulates user input for testing purposes
sample_input = {
    "Delta_TP9": 0.290123, "Delta_AF7": 0.045678, "Delta_AF8": 0.00456, "Delta_TP10": 0.200789,
        "Theta_TP9": 0.300345, "Theta_AF7": -0.180234, "Theta_AF8": 0.140567, "Theta_TP10": 0.200456,
        "Alpha_TP9": 0.710789, "Alpha_AF7": 0.097890, "Alpha_AF8": 0.125678, "Alpha_TP10": 0.600456,
        "Beta_TP9": 0.310234, "Beta_AF7": 0.007890, "Beta_AF8": -0.025789, "Beta_TP10": 0.250345,
        "Gamma_TP9": 0.105678, "Gamma_AF7": -0.220789, "Gamma_AF8": -0.300567, "Gamma_TP10": 0.240567,
        "RAW_TP9": 745.67890, "RAW_AF7": 735.12345, "RAW_AF8": 725.23456, "RAW_TP10": 740.34567
}

# Here, we simulate the input by using the sample_input dictionary
for feature in features:
    # Replace the input method with our sample_input for testing
    predict[feature] = [sample_input[feature]]


predictions = classifier.predict(input_fn=lambda: input_fn(predict))


for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]  # Get the predicted class ID
    probability = pred_dict['probabilities'][class_id]  # Get the associated probability

    # Only print prediction if probability is greater than 0.8
    if probability > 0.8:
        print('Prediction is "{}" ({:.1f}%)'.format(
            SPECIES.get(class_id, 'Unknown'), 100 * probability))
    else:
        print("Prediction probability is below threshold (0.8). No prediction made.")


Prediction is "Yes" (87.4%)
